In [2]:
import logging
import traceback

import pandas as pd
import xlwings as xw
from sixgill.definitions import *
from sixgill.pipesim import Model, Units

from network_simulation import NetworkSimulation, dict_to_df

# Constants
FOLDER_DIRECTORY = r"c:\Users\IDM252577\Desktop\Python Projects\PIPSIM"
MODEL_FILENAME = "BAB HP.pips"
EXCEL_FILE = "Water Injection Profile for BAB HP.xlsx"

SOURCE_NAME: str = "Sea Water Supply"
PUMP_NAME: list = ["HP Pump", "HP PUMP 2"]

ns = NetworkSimulation(FOLDER_DIRECTORY, MODEL_FILENAME, EXCEL_FILE)

for case in ["C1"]:
    for condition in ["W-LP-EO"]:
        try:
            ns.run_app(source_name=SOURCE_NAME, pump_name=PUMP_NAME)
        except:
            print(traceback.format_exc())
            continue

INFO:network_simulation:------------Network Simulation Object Created----------------

INFO:network_simulation:case not specified, using C1
INFO:network_simulation:condition not specified, using W-LP-EO
INFO:manta.server.manager:Starting PIPESIM server on thread ID: 1904
INFO:manta.server.manager:Python toolkit license is checked out successfully
INFO:manta.server.manager:Waiting for PIPESIM server to start.
INFO:sixgill.core.metadata:Using cached metadata for 'http://localhost:60667/api/metadata#'
INFO:network_simulation:Model prepared
 Case:C1
 Condition: W-LP-EO
INFO:network_simulation:Set Global Conditions
INFO:network_simulation:Activated all wells
INFO:network_simulation:Deactivated no flow wells
INFO:network_simulation:Populated flowrates in model from excel
INFO:network_simulation:Model saved as c:\Users\IDM252577\Desktop\Python Projects\PIPSIM\C1_W-LP-EO_BAB HP.pips
INFO:sixgill.core.run_or_start_simulation:Simulation is running.
INFO:sixgill.core.run_or_start_simulation:Simul

Traceback (most recent call last):
  File "C:\Users\IDM252577\AppData\Local\Temp\ipykernel_11276\3332138964.py", line 24, in <module>
    ns.run_app(source_name=SOURCE_NAME, pump_name=PUMP_NAME)
  File "c:\Users\IDM252577\Desktop\Python Projects\PIPSIM\network_simulation.py", line 258, in run_app
    raise e
  File "c:\Users\IDM252577\Desktop\Python Projects\PIPSIM\network_simulation.py", line 251, in run_app
    self.write_results_to_excel()
  File "c:\Users\IDM252577\Desktop\Python Projects\PIPSIM\network_simulation.py", line 214, in write_results_to_excel
    df=self.node_results,
       ^^^^^^^^^^^^^^^^^
AttributeError: 'NetworkSimulation' object has no attribute 'node_results'



In [5]:
ns.results.node

defaultdict(collections.OrderedDict,
            {'Pressure': OrderedDict([('Unit', 'psia'),
                          ('J 111', 3286.71493586253),
                          ('J 109', 3298.7389914111022),
                          ('J 115', 3290.901614327867),
                          ('IWS004', 3291.6595373597274),
                          ('BB-1072', 3269.5731833135064),
                          ('J 53', 3308.3361198245816),
                          ('J 95', 3185.44649301788),
                          ('HB1IW-002', 3228.286672392094),
                          ('HB1IW-007', 3226.772174869341),
                          ('J 274', 3271.6262387710008),
                          ('HB1IW-005', 3184.91046535676),
                          ('J 263', 3242.362425475143),
                          ('IWW003', 3252.6645479260337),
                          ('J 27', 3289.8950233298656),
                          ('J 275', 3166.507012020336),
                          ('J 110', 3304.589098145